## This is a tutorial notebook where a car passes a slower front car
- Flow
    - Create roadway and initial scene by placing cars on the roadway
    - Associate driver models to the vehicles
    - Run a simulation
    - Create an associated video and save to media folder

In [ ]:
using AutomotiveDrivingModels
using AutoViz
using Reel

# Useful helper functions

In [ ]:
# function: generate roadway and place cars
"""
    function init_place_cars(lane_place_array;road_length = 400.0)
- Place cars on a straight roadway of `road_length` according to elems in `lane_place_array`

# Examples
```julia
pos_vel_array_1 = [(200.,30.),(215.,0.),(220.,0.)]
pos_vel_array_2 = [(200.,0.),(215.,0.),(220.,20.)]
pos_vel_array_3 = [(215.,0.),(225.,10.),(230.,0.)]
lane_place_array = [pos_vel_array_1,pos_vel_array_2,pos_vel_array_3]
scene,roadway = init_place_cars(lane_place_array)
```
"""
function init_place_cars(lane_place_array;road_length = 400.0)
    num_lanes = length(lane_place_array)
    roadway = gen_straight_roadway(num_lanes,road_length)
    scene = Scene()

    id = 1
    for i in 1:num_lanes
        for j in 1:length(lane_place_array[i])
            veh_state = VehicleState(Frenet(roadway[LaneTag(1,i)],
                    lane_place_array[i][j][1]),roadway,
                lane_place_array[i][j][2])
            veh = Vehicle(veh_state,VehicleDef(),id)
            push!(scene,veh)
            id+=1
        end
    end
    return scene,roadway
end

In [ ]:
# function: hallucinate scene list
"""
    function get_hallucination_scenes
- Hallucinate starting from `start_step` for `nsteps` using `models` and return a list of scenes
- Used by `plot_carwise_pos_vel` to assess position and velocity traces against ground truth

# Returns
- `halluc_scenes_list`: List containing the scenes starting with the ground truth scene at `start_step`

# Examples
```julia
true_scene_list = get_hallucination_scenes(scene_halluc,duration=10,models=models);
```
"""
function get_hallucination_scenes(start_scene;models,start_step=1,duration=10,
        id_list=[],verbosity = false,timestep=TIMESTEP)
        # Setting up
    scene_halluc = start_scene
    halluc_scenes_list = []
    push!(halluc_scenes_list,deepcopy(start_scene))
#     scene_halluc = get_scene(start_step,traj) # Frame to start hallucination from
#     push!(halluc_scenes_list,deepcopy(scene_halluc))
    
    nsteps = duration/timestep
    for (i,t) in enumerate(start_step:start_step+nsteps-1)
        
#         if !isempty(id_list) keep_vehicle_subset!(scene_halluc,id_list) end
        
        actions = Array{Any}(undef,length(scene_halluc))

            # Propagation of scene forward
        get_actions!(actions,scene_halluc,ROADWAY,models)

        tick!(scene_halluc,ROADWAY,actions,timestep)
        
        push!(halluc_scenes_list,deepcopy(scene_halluc))
    end 
    return halluc_scenes_list
end

In [ ]:
# function: make a video from a list of scenes
"""
    function scenelist2video(scene_list;filename = "media/mobil/scene_to_video.mp4")
- Make video from a list of scenes

# Examples
```julia
scene_list = get_hallucination_scenes(scene2,models=models,traj=int_trajdata,roadway=roadway2)
scenelist2video(scene_list,roadway=roadway2,filename="media/interaction_vids/merge.mp4")
```
"""
function scenelist2video(scene_list;id_list=[],
        filename = "media/mobil/scene_to_video.mp4",roadway=roadway_interaction)
    frames = Frames(MIME("image/png"),fps = 10)
    
    # Loop over list of scenes and convert to video
    for i in 1:length(scene_list)
        if !isempty(id_list) keep_vehicle_subset!(scene_list[i],id_list) end
        scene_visual = render(scene_list[i],roadway,
        #cam=FitToContentCamera(1.), # uncomment this if you want to fit the entire roadway on screen
        cam = CarFollowCamera(1)
        )
        push!(frames,scene_visual)
    end
    print("Making video filename: $(filename)\n")
    write(filename,frames)
    return nothing
end

## Okay, functions have been defined. Let's actually use them in a simulation

In [ ]:
# Start the scene by placing vehicles
pos_vel_array_1 = [(100.,30.),(150.,0.)]
pos_vel_array_2 = [(150.,10.)] #(280.,10.)

lane_place_array = [pos_vel_array_1,pos_vel_array_2]
scene,roadway = init_place_cars(lane_place_array)
const TIMESTEP = 0.1;
const SCENE = deepcopy(scene)
const ROADWAY = roadway;

In [ ]:
start_step=1
nsteps=100
scene_halluc = deepcopy(SCENE)
models = Dict{Int64,DriverModel}()
for veh in scene models[veh.id] = IntelligentDriverModel() end
models[1] = Tim2DDriver(TIMESTEP,
                        mlane=MOBIL(TIMESTEP),
                        mlon=IntelligentDriverModel(),
            )
models[2] = IntelligentDriverModel(v_des=15.)
models[3] = IntelligentDriverModel()

true_scene_list = get_hallucination_scenes(scene_halluc,duration=10,models=models);
scenelist2video(true_scene_list,roadway=ROADWAY,filename="media/tutorial.mp4")